In [ ]:
# !pip install psycopg2-binary


In [ ]:
import psycopg2

# Define your database connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': '5432'  # Default port for PostgreSQL
}

# Establish the connection
try:
    connection = psycopg2.connect(**db_params)
    print("Connection to PostgreSQL database successful!")
except Exception as e:
    print(f"An error occurred: {e}")

Connection to PostgreSQL database successful!


In [16]:
import pandas as pd

# Query data from three different tables
try:
    query = """
    SELECT * FROM public.api_question
    """
    df = pd.read_sql_query(query, connection)
    print("Data fetched successfully!")
    print(df.head())  # Display the first few rows of the dataframe
except Exception as e:
    print(f"An error occurred while fetching data: {e}")

Data fetched successfully!
                        created_at                       updated_at  \
0 2025-03-25 11:07:01.173446+00:00 2025-04-02 13:03:51.081200+00:00   
1 2025-03-25 11:07:01.182221+00:00 2025-04-02 13:03:51.098349+00:00   
2 2025-03-25 11:07:01.345035+00:00 2025-04-02 13:03:51.324499+00:00   
3 2025-03-25 11:07:01.353563+00:00 2025-04-02 13:03:51.339643+00:00   
4 2025-03-25 11:07:01.362914+00:00 2025-04-02 13:03:51.355353+00:00   

           id                                      question_text passage  \
0  m_pdiunnex    "Solve for ##x## in the equation: ##3x = 15##."           
1  m_5ixa05lt  "Solve for ##x## in the equation: ##x - 4 = 9##."           
2  m_niqm0fgn  "A line passes through the points ##(3, 4)## a...           
3  m_3xtl6iay  "A line passes through the point (3, 4) and is...           
4  m_7nntk3pe  "Two lines are given by the equations ##3x + 4...           

  question_type                                          rationale  \
0           MCQ  {'

C:\Users\Manideep S\AppData\Local\Temp\ipykernel_18900\1489443608.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [17]:
df = df[['question_text', 'passage', 'question_type', 'rationale', 'options', 'correct_answer', 'hint', 'prompt_id']]
print(df.head())

                                       question_text passage question_type  \
0    "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
1  "Solve for ##x## in the equation: ##x - 4 = 9##."                   MCQ   
2  "A line passes through the points ##(3, 4)## a...                   MCQ   
3  "A line passes through the point (3, 4) and is...                   MCQ   
4  "Two lines are given by the equations ##3x + 4...                   MCQ   

                                           rationale  \
0  {'reasoning': 'To solve for ##x##, divide both...   
1  {'reasoning': 'To solve for ##x##, add 4 to bo...   
2  {'reasoning': 'First, calculate the slope ##m ...   
3  {'reasoning': 'The slope of the line ##2x - 3y...   
4  {'reasoning': 'For two lines to be parallel, t...   

                                             options correct_answer  \
0  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            [D]   
1  [{'choice_id': 'A', 'choice_text': '##x## = 5'...

In [18]:
# Query to fetch prompt data based on the prompt_id
try:
    prompt_query = """
    SELECT p.name, p.id, p.question_prompt_id
    FROM public.api_prompt p
    INNER JOIN public.api_question q ON p.id = q.prompt_id
    """
    # Extract unique prompt_ids from the dataframe
    prompt_ids = tuple(df['prompt_id'].unique())
    
    # Execute the query
    prompt_df = pd.read_sql_query(prompt_query, connection, params=(prompt_ids,))
    print("Prompt data fetched successfully!")
    print(prompt_df.head())  # Display the first few rows of the prompt dataframe
except Exception as e:
    print(f"An error occurred while fetching prompt data: {e}")

Prompt data fetched successfully!
                    name                                    id  \
0  PID-M-ALG-LEO-EMT-EAS  ca139627-ed04-4b12-97e8-0486f28beb07   
1  PID-M-ALG-LEO-EMT-EAS  ca139627-ed04-4b12-97e8-0486f28beb07   
2  PID-M-ALG-LIF-EMT-MED  b6e58fc7-56aa-4ef7-9c46-ef25b0591169   
3  PID-M-ALG-LIF-EMT-HAR  7b2357b7-eca7-4b4e-aa39-7257cf8258fa   
4  PID-M-ALG-LIF-EMT-HAR  7b2357b7-eca7-4b4e-aa39-7257cf8258fa   

                     question_prompt_id  
0  d721b009-a941-482e-8ea7-22e6d515681f  
1  d721b009-a941-482e-8ea7-22e6d515681f  
2  ad052292-cf26-4009-bde6-faaa9e1cd007  
3  9bc5ab8c-96e4-420e-8e28-41d5bfbe334d  
4  9bc5ab8c-96e4-420e-8e28-41d5bfbe334d  


C:\Users\Manideep S\AppData\Local\Temp\ipykernel_18900\3681442312.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prompt_df = pd.read_sql_query(prompt_query, connection, params=(prompt_ids,))


In [19]:
# Merge df with prompt_df on 'prompt_id' and 'id'
df = df.merge(prompt_df[['id', 'name','question_prompt_id']], left_on='prompt_id', right_on='id', how='left')

In [20]:


# Rename the 'name' column to 'prompt'
df.rename(columns={'name': 'prompt_name'}, inplace=True)

# df.rename(columns={'question_pron': 'prompt_name'}, inplace=True)

# Drop the redundant 'id' column from the merge
df.drop(columns=['id'], inplace=True)

print(df.head())

                                     question_text passage question_type  \
0  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
1  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
2  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
3  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
4  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   

                                           rationale  \
0  {'reasoning': 'To solve for ##x##, divide both...   
1  {'reasoning': 'To solve for ##x##, divide both...   
2  {'reasoning': 'To solve for ##x##, divide both...   
3  {'reasoning': 'To solve for ##x##, divide both...   
4  {'reasoning': 'To solve for ##x##, divide both...   

                                             options correct_answer  \
0  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            [D]   
1  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            

In [21]:
df.head()

,question_text,passage,question_type,rationale,options,correct_answer,hint,prompt_id,prompt_name,question_prompt_id
0,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f
1,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f
2,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f
3,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f
4,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f


In [22]:
columns = prompt_df.columns.tolist()
print(columns)

['name', 'id', 'question_prompt_id']


In [23]:
columns = df.columns.tolist()
print(columns)

['question_text', 'passage', 'question_type', 'rationale', 'options', 'correct_answer', 'hint', 'prompt_id', 'prompt_name', 'question_prompt_id']


In [24]:
# Query to fetch data from the api_questionprompt table
try:
    question_prompt_query = """
    SELECT id, domain, skill, subject, difficulty
    FROM public.api_questionprompt
    """
    
    # Fetch the data into a dataframe
    question_prompt_df = pd.read_sql_query(question_prompt_query, connection)
    print("Question prompt data fetched successfully!")
    print(question_prompt_df.head())  # Display the first few rows of the question prompt dataframe
except Exception as e:
    print(f"An error occurred while fetching or merging question prompt data: {e}")

Question prompt data fetched successfully!
                                     id               domain        skill  \
0  afd788d1-4c72-4745-aeea-a41230a612e4  Expression of Ideas  Transitions   
1  a27763c9-5c7c-4035-9c4a-9ab431d05bfb  Expression of Ideas  Transitions   
2  b6190818-eb33-4708-9af7-69247da0c62b  Expression of Ideas  Transitions   
3  cdaa48fd-f6fd-4fcf-909c-7630de2066f1  Expression of Ideas  Transitions   
4  6bd33fa2-fa52-4f3b-9efc-e25f7beac32e  Expression of Ideas  Transitions   

  subject difficulty  
0      RW        MED  
1      RW        VHA  
2      RW        VHA  
3      RW        VHA  
4      RW        HAR  


C:\Users\Manideep S\AppData\Local\Temp\ipykernel_18900\1716527989.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  question_prompt_df = pd.read_sql_query(question_prompt_query, connection)


In [25]:
question_prompt_df['subject'] = question_prompt_df['subject'].replace({'RW': 'Read Write'})

In [26]:
question_prompt_df.head()

,id,domain,skill,subject,difficulty
0,afd788d1-4c72-4745-aeea-a41230a612e4,Expression of Ideas,Transitions,Read Write,MED
1,a27763c9-5c7c-4035-9c4a-9ab431d05bfb,Expression of Ideas,Transitions,Read Write,VHA
2,b6190818-eb33-4708-9af7-69247da0c62b,Expression of Ideas,Transitions,Read Write,VHA
3,cdaa48fd-f6fd-4fcf-909c-7630de2066f1,Expression of Ideas,Transitions,Read Write,VHA
4,6bd33fa2-fa52-4f3b-9efc-e25f7beac32e,Expression of Ideas,Transitions,Read Write,HAR


In [ ]:
question_prompt_df["difficulty"] = question_prompt_df["difficulty"].replace({
	EAS: "Easy",
	MED: "Medium",
	HAR: "Hard",
	VHA: "Very Hard",
});


In [ ]:
question_prompt_df.head();


,id,domain,skill,subject,difficulty
0,afd788d1-4c72-4745-aeea-a41230a612e4,Expression of Ideas,Transitions,Read Write,Medium
1,a27763c9-5c7c-4035-9c4a-9ab431d05bfb,Expression of Ideas,Transitions,Read Write,Very Hard
2,b6190818-eb33-4708-9af7-69247da0c62b,Expression of Ideas,Transitions,Read Write,Very Hard
3,cdaa48fd-f6fd-4fcf-909c-7630de2066f1,Expression of Ideas,Transitions,Read Write,Very Hard
4,6bd33fa2-fa52-4f3b-9efc-e25f7beac32e,Expression of Ideas,Transitions,Read Write,Hard


In [30]:

    
# Perform a left join with df on question_prompt_id and id
df = df.merge(question_prompt_df, left_on='question_prompt_id', right_on='id', how='left')

# Drop the redundant 'id' column from the merge
df.drop(columns=['id'], inplace=True)

print("Dataframe after merging:")
print(df.head())  # Display the first few rows of the updated dataframe


Dataframe after merging:
                                     question_text passage question_type  \
0  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
1  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
2  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
3  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
4  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   

                                           rationale  \
0  {'reasoning': 'To solve for ##x##, divide both...   
1  {'reasoning': 'To solve for ##x##, divide both...   
2  {'reasoning': 'To solve for ##x##, divide both...   
3  {'reasoning': 'To solve for ##x##, divide both...   
4  {'reasoning': 'To solve for ##x##, divide both...   

                                             options correct_answer  \
0  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            [D]   
1  [{'choice_id': 'A', 'choice_text': '

In [31]:
df.head()

,question_text,passage,question_type,rationale,options,correct_answer,hint,prompt_id,prompt_name,question_prompt_id,domain,skill,subject,difficulty
0,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f,Algebra,Linear equations in one variable,Math,Easy
1,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f,Algebra,Linear equations in one variable,Math,Easy
2,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f,Algebra,Linear equations in one variable,Math,Easy
3,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f,Algebra,Linear equations in one variable,Math,Easy
4,"""Solve for ##x## in the equation: ##3x = 15##.""",,MCQ,"{'reasoning': 'To solve for ##x##, divide both...","[{'choice_id': 'A', 'choice_text': '##x## = 3'...",[D],Consider what operation will isolate ##x##.,ca139627-ed04-4b12-97e8-0486f28beb07,PID-M-ALG-LEO-EMT-EAS,d721b009-a941-482e-8ea7-22e6d515681f,Algebra,Linear equations in one variable,Math,Easy


In [32]:
Embedding_DF = df[['question_text', 'passage', 'question_type', 'rationale', 'options', 'correct_answer', 'hint', 'domain', 'skill', 'subject', 'difficulty']]

In [34]:
print(Embedding_DF.head())

                                     question_text passage question_type  \
0  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
1  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
2  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
3  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
4  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   

                                           rationale  \
0  {'reasoning': 'To solve for ##x##, divide both...   
1  {'reasoning': 'To solve for ##x##, divide both...   
2  {'reasoning': 'To solve for ##x##, divide both...   
3  {'reasoning': 'To solve for ##x##, divide both...   
4  {'reasoning': 'To solve for ##x##, divide both...   

                                             options correct_answer  \
0  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            [D]   
1  [{'choice_id': 'A', 'choice_text': '##x## = 3'...            

In [35]:
columns = Embedding_DF.columns.tolist()
print(columns)

['question_text', 'passage', 'question_type', 'rationale', 'options', 'correct_answer', 'hint', 'domain', 'skill', 'subject', 'difficulty']


In [37]:
# Flatten JSON columns and convert them to strings
flattened_df = Embedding_DF.applymap(lambda x: str(x) if isinstance(x, (dict, list)) else x)

# Create a new DataFrame with the flattened data
new_df = pd.DataFrame(flattened_df)

print(new_df.head())

                                     question_text passage question_type  \
0  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
1  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
2  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
3  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   
4  "Solve for ##x## in the equation: ##3x = 15##."                   MCQ   

                                           rationale  \
0  {'reasoning': 'To solve for ##x##, divide both...   
1  {'reasoning': 'To solve for ##x##, divide both...   
2  {'reasoning': 'To solve for ##x##, divide both...   
3  {'reasoning': 'To solve for ##x##, divide both...   
4  {'reasoning': 'To solve for ##x##, divide both...   

                                             options correct_answer  \
0  [{'choice_id': 'A', 'choice_text': '##x## = 3'...          ['D']   
1  [{'choice_id': 'A', 'choice_text': '##x## = 3'...          ['

C:\Users\Manideep S\AppData\Local\Temp\ipykernel_18900\2121595242.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  flattened_df = Embedding_DF.applymap(lambda x: str(x) if isinstance(x, (dict, list)) else x)


In [38]:
# Concatenate all columns into a single column
concatenated_df = new_df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1).to_frame(name='concatenated_column')

print(concatenated_df.head())

                                 concatenated_column
0  "Solve for ##x## in the equation: ##3x = 15##....
1  "Solve for ##x## in the equation: ##3x = 15##....
2  "Solve for ##x## in the equation: ##3x = 15##....
3  "Solve for ##x## in the equation: ##3x = 15##....
4  "Solve for ##x## in the equation: ##3x = 15##....


In [39]:
import re

# Function to clean text
def clean_text(text):
    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the cleaning function to the concatenated column
concatenated_df['cleaned_text'] = concatenated_df['concatenated_column'].apply(clean_text)

print(concatenated_df.head())

                                 concatenated_column  \
0  "Solve for ##x## in the equation: ##3x = 15##....   
1  "Solve for ##x## in the equation: ##3x = 15##....   
2  "Solve for ##x## in the equation: ##3x = 15##....   
3  "Solve for ##x## in the equation: ##3x = 15##....   
4  "Solve for ##x## in the equation: ##3x = 15##....   

                                        cleaned_text  
0  solve for x in the equation x mcq reasoning to...  
1  solve for x in the equation x mcq reasoning to...  
2  solve for x in the equation x mcq reasoning to...  
3  solve for x in the equation x mcq reasoning to...  
4  solve for x in the equation x mcq reasoning to...  


In [41]:
# Remove ## and [, ] from the text
concatenated_df['cleaned_text'] = concatenated_df['cleaned_text'].str.replace(r'##', '', regex=True)
concatenated_df['cleaned_text'] = concatenated_df['cleaned_text'].str.replace(r'[\[\]]', '', regex=True)

print(concatenated_df.head())

                                 concatenated_column  \
0  "Solve for ##x## in the equation: ##3x = 15##....   
1  "Solve for ##x## in the equation: ##3x = 15##....   
2  "Solve for ##x## in the equation: ##3x = 15##....   
3  "Solve for ##x## in the equation: ##3x = 15##....   
4  "Solve for ##x## in the equation: ##3x = 15##....   

                                        cleaned_text  
0  solve for x in the equation x mcq reasoning to...  
1  solve for x in the equation x mcq reasoning to...  
2  solve for x in the equation x mcq reasoning to...  
3  solve for x in the equation x mcq reasoning to...  
4  solve for x in the equation x mcq reasoning to...  


In [ ]:
!pip install sentence-transformers

In [42]:
from sentence_transformers import SentenceTransformer
import numpy as np

# !pip install sentence-transformers


# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Perform embeddings on the 'cleaned_text' column
concatenated_df['embeddings'] = concatenated_df['cleaned_text'].apply(lambda x: model.encode(x))

# Convert embeddings to a NumPy array for further processing
embeddings_array = np.array(concatenated_df['embeddings'].tolist())

print("Embeddings generated successfully!")
print(embeddings_array.shape)  # Display the shape of the embeddings array


Embeddings generated successfully!
(6835, 384)


In [43]:
print(embeddings_array[0])

[-2.21707560e-02  1.96309574e-02  4.90479842e-02  4.39473838e-02
 -2.33131764e-03  2.71468945e-02  4.83601615e-02 -8.15802068e-03
  1.81793850e-02  5.96189611e-02  5.92922047e-02 -7.95385465e-02
  5.12700826e-02  7.38034844e-02  4.69194017e-02  6.09528385e-02
 -1.00210616e-02  5.43692224e-02 -5.30625135e-02  5.51956072e-02
 -4.93427878e-03 -5.96360974e-02 -8.22769031e-02  1.55087588e-02
  4.93276045e-02  1.02958180e-01  3.83117720e-02 -7.98170567e-02
  4.93008532e-02 -1.76712759e-02  2.75570638e-02  1.08520754e-01
  9.57151279e-02 -5.10557778e-02 -4.07200083e-02 -1.34923579e-02
 -2.66132876e-02 -3.78975458e-02 -2.49466281e-02 -5.91410585e-02
 -7.15057999e-02 -8.59111175e-03  2.32682360e-04  1.37147974e-04
 -2.31679506e-03 -1.85909718e-02  5.14801219e-03 -4.44415733e-02
  3.29178050e-02  9.88466218e-02  4.61481549e-02  4.34631109e-02
 -1.31287664e-01 -1.45708248e-02  1.21682301e-01  1.98336914e-02
 -7.57032335e-02  2.19671093e-02 -9.76688112e-04 -4.05806536e-03
 -8.47268924e-02  5.05478